In [1]:
import os
import glob
import pandas as pd
import numpy as np 
import pingouin as pg 
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")


In [2]:
# One-sample t-test to evaluate that is there obvious possitve/negative correlation Dynamic rating and Eye-factors 

curr_dir = os.getcwd()
avg_dir = curr_dir +"/AVG" 

data_list = os.listdir(avg_dir)
data_list = sorted([ data for data in data_list if data.endswith(".csv") and not data.startswith(".") and data != 'Dynamic_Rating.csv'])


avg_data = { str(x).split(sep=".")[0] : pd.read_csv(f"{avg_dir}/{x}", index_col=0) for x in data_list}
avg_data.keys()


dict_keys(['Blinking', 'Fixation_Duration', 'Fixation_Num', 'Saccade_Duration', 'left_iris', 'left_pupil', 'right_iris', 'right_pupil'])

In [3]:
of_list= os.listdir(curr_dir + '/Feature')
v_list = os.listdir(curr_dir + '/HSV')

of_list = sorted([ int(x.split(sep ='_')[0]) for x in of_list])
v_list = sorted([ int(x.split(sep ='_')[0]) for x in v_list])

of_data = np.array([np.load( curr_dir + '/Feature/' + str(of) + '_feature.npy')[:3600] for of in of_list])
v_data = np.array([np.load(curr_dir + '/HSV/' + str(v) + '_v_values.npy')[:3600] for v in v_list])

print(of_data.shape)
print(v_data.shape)

(16, 3600)
(16, 3600, 256, 256)


In [4]:
of_data = of_data.mean(axis=1)
v_data = v_data.mean(axis=(1,2,3))

print(len(of_data))
print(len(v_data))

16
16


In [5]:
def get_correaltion( arr1, arr2 ) :
    df = pg.corr( arr1, arr2, method='spearman')
    
    r = df['r'][0] if not np.isnan(df['r'][0]) else 0
    p = df["p-val"][0]
    return r, p

def get_cor_by_participants( arr1, arr2 ) :
    
    r_values = []
    for ind in range(len(arr1)) : 
        r, p = get_correaltion( arr1.iloc[ind,:], arr2.iloc[ind,:])
        r_values.append(r)
    
    return r_values

def get_cor_with_video( arr1, arr2 ) :
    
    r_values = []
    for ind in range(len(arr1)) : 
        r, p = get_correaltion( arr1.iloc[ind,:], arr2)
        r_values.append(r)
    
    return r_values 

def one_sample_t_test( arr, compared_value = 0 ) : 
    df = pg.ttest(arr, compared_value)

    return df["T"][0], df["p-val"][0]

In [6]:
dynamic_rating = pd.read_csv( avg_dir + "/Dynamic_Rating.csv", index_col=0)

for data in avg_data : 
    r_list = get_cor_by_participants( avg_data[data], dynamic_rating)
    t, p = one_sample_t_test(r_list)

    print(data)
    print("-"*50)
    print("참가자별 r 값 :")
    print(np.round(r_list, 2))
    print("r 평균값 :", np.mean(r_list).round(2))
    print("-"*50)
    print("1sample t-test : t = {0}, p = {1}".format(round(t,2),round(p,2)))
    print("="*50, '\n')

Blinking
--------------------------------------------------
참가자별 r 값 :
[ 0.16  0.    0.33  0.03 -0.1  -0.71 -0.03  0.53  0.72  0.63 -0.14 -0.01
 -0.28  0.11 -0.14  0.28  0.18 -0.19  0.15  0.19 -0.1  -0.24  0.11  0.
  0.22  0.21  0.17 -0.43  0.31 -0.16  0.36 -0.05  0.57 -0.08  0.42  0.44
 -0.06  0.37 -0.28  0.66  0.03 -0.36  0.29  0.31  0.34 -0.49 -0.08  0.57
  0.   -0.22 -0.34 -0.31]
r 평균값 : 0.07
--------------------------------------------------
1sample t-test : t = 1.68, p = 0.1

Fixation_Duration
--------------------------------------------------
참가자별 r 값 :
[-0.65  0.   -0.29 -0.05 -0.51 -0.26 -0.33  0.   -0.49  0.16 -0.35 -0.15
 -0.21 -0.12 -0.1  -0.69 -0.74  0.51  0.06  0.   -0.49 -0.12 -0.36  0.
 -0.41 -0.66  0.1  -0.52 -0.03 -0.36 -0.59 -0.16 -0.46 -0.36 -0.28  0.55
 -0.05 -0.09  0.07 -0.26 -0.18  0.51 -0.37 -0.63  0.2   0.29 -0.38 -0.57
 -0.4  -0.3  -0.14  0.15]
r 평균값 : -0.2
--------------------------------------------------
1sample t-test : t = -4.72, p = 0.0

Fixation_Num
---

In [30]:
for data in avg_data : 

    r_list = get_cor_with_video( avg_data[data], of_data)
    t, p = one_sample_t_test(r_list)

    print(data)
    print("-"*50)
    # print("참가자별 r 값 :", np.round(r_list, 2))
    # print("-"*50)
    print("1sample t-test : t = {0}, p = {1}".format(round(t,2),round(p,2)))
    print("="*50, '\n')

r_list = get_cor_with_video( dynamic_rating, of_data)
t, p = one_sample_t_test(r_list)
print("DynamicRating")
print("-"*50)
# print("참가자별 r 값 :", np.round(r_list, 2))
# print("-"*50)
print("1sample t-test : t = {0}, p = {1}".format(round(t,2),round(p,2)))
print("="*50, '\n')

Blinking
--------------------------------------------------
1sample t-test : t = -2.99, p = 0.0

Fixation_Duration
--------------------------------------------------
1sample t-test : t = -7.78, p = 0.0

Fixation_Num
--------------------------------------------------
1sample t-test : t = 10.77, p = 0.0

Saccade_Duration
--------------------------------------------------
1sample t-test : t = -5.51, p = 0.0

left_iris
--------------------------------------------------
1sample t-test : t = 14.76, p = 0.0

left_pupil
--------------------------------------------------
1sample t-test : t = -7.35, p = 0.0

right_iris
--------------------------------------------------
1sample t-test : t = 13.87, p = 0.0

right_pupil
--------------------------------------------------
1sample t-test : t = -7.31, p = 0.0

DynamicRating
--------------------------------------------------
1sample t-test : t = 7.45, p = 0.0



In [29]:
for data in avg_data : 

    r_list = get_cor_with_video( avg_data[data], v_data)
    t, p = one_sample_t_test(r_list)

    print(data)
    print("-"*50)
    # print("참가자별 r 값 :", np.round(r_list, 2))
    # print("-"*50)
    print("1sample t-test : t = {0}, p = {1}".format(round(t,2),round(p,2)))
    print("="*50, '\n')

r_list = get_cor_with_video( dynamic_rating, v_data)
t, p = one_sample_t_test(r_list)
print("DynamicRating")
print("-"*50)
# print("참가자별 r 값 :", np.round(r_list, 2))
# print("-"*50)
print("1sample t-test : t = {0}, p = {1}".format(round(t,2),round(p,2)))
print("="*50, '\n')

Blinking
--------------------------------------------------
1sample t-test : t = -1.1, p = 0.28

Fixation_Duration
--------------------------------------------------
1sample t-test : t = -5.37, p = 0.0

Fixation_Num
--------------------------------------------------
1sample t-test : t = 7.9, p = 0.0

Saccade_Duration
--------------------------------------------------
1sample t-test : t = -4.08, p = 0.0

left_iris
--------------------------------------------------
1sample t-test : t = 29.74, p = 0.0

left_pupil
--------------------------------------------------
1sample t-test : t = -19.67, p = 0.0

right_iris
--------------------------------------------------
1sample t-test : t = 26.21, p = 0.0

right_pupil
--------------------------------------------------
1sample t-test : t = -16.52, p = 0.0

DynamicRating
--------------------------------------------------
1sample t-test : t = 4.52, p = 0.0

